# Machine learning for neuroimaging with scikit-learn

Reference:  
[1\] https://www.frontiersin.org/articles/10.3389/fninf.2014.00014/full#B13   
[2\] https://www.datacamp.com/community/tutorials/reconstructing-brain-images-deep-learning
  

In [8]:
# ===== # IMPORTING # ===== #
# [1] 
import sklearn
import nibabel as nib
import numpy as np
from matplotlib import pyplot as plt

# Also load the utility functions
import sys
import os
PWD = os.path.dirname(os.path.abspath(os.getcwd()))
NOTEBOOK_PATH = os.path.abspath(os.path.join(PWD, 'notebooks'))
UTILS_PATH = os.path.abspath(os.path.join(PWD, 'utils'))
sys.path.append(UTILS_PATH)

%run ../utils/datasets.py
%run ../utils/masking.py
%run ../utils/signal.py

# [2]
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"       # model will be trained on GPU 1

import cv2
from keras.layers import (
  Input, Dense, Flatten, Dropout, merge, 
  Reshape, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
)
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K

print(K.tensorflow_backend._get_available_gpus())

import scipy.misc
import numpy.random as rng
from PIL import (Image, ImageDraw, ImageFont)
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import math
import glob

%matplotlib inline
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard module is not an IPython extension.


In [4]:
# ===== # FETCHING DATA # ===== #

dataset = fetch_miyawaki2008(data_dir='../data')
# %run ../data/miyawaki2008/convert_miyawaki.py

In [ ]:
# ===== # LOADING DATA # ===== #

# Keep only random runs
X_random = dataset.func[12:]
y_random = dataset.label[12:]
y_shape = (10, 10)

X_train = []

for x_rand in X_random:
  # Mask data
  x_img = nibabel.load(x_rand)
  x = apply_mask(x_img, dataset.mask)
  x = clean(x, standardize=True, detrend=True)
  X_train.append(x)

In [ ]:
print(len(X_train))
print(X_train[0].shape)
print(X_train[0][0,:])

In [ ]:
# %run ../utils/visualization_101.py

# Load image
bg_img = nibabel.load('../utils/bg.nii.gz')
print(bg_img)
bg = bg_img.get_data()
# Keep values over 4000 as activation map
act = bg.copy()
act[act < 6000] = 0.

# Display the background
plt.imshow(bg[..., 10].T, origin='lower', interpolation='nearest', cmap='gray')
# Mask background values of activation map
masked_act = np.ma.masked_equal(act, 0.)
plt.imshow(masked_act[..., 10].T, origin='lower', interpolation='nearest', cmap='hot')
# Cosmetics: disable axis
plt.axis('off')
plt.show()
# Save the activation map
# nibabel.save(nibabel.Nifti1Image(act, bg_img.affine), 'activation.nii.gz')